************
# <font color=#7451eb><center> Notebook du nettoyage des données</center></font>
## <font color=#7451eb><center> PROJET n° 3 : Concevez une application au service de la santé publique</center></font>
************   

## <font color=#7451eb>CONTEXTE</font> :

L'agence **"Santé publique France"** a lancé un appel à **projets** pour trouver des **idées innovantes d’applications** en  lien avec l'**alimentation**. Vous souhaitez y participer et **proposer une idée d’application**.
<br>
**********
## <font color=#7451eb>DATASET</font> :

Le **Dataset - Open Food Fact** est disponible sur le site officiel : (https://world.openfoodfacts.org/). <br> 
Les **variables** sont définies à cette adresse : (https://world.openfoodfacts.org/data/data-fields.txt)

Les champs sont séparés en quatre sections :

* Les informations générales sur la fiche du produit : nom, date de modification, etc.<br>
* Un ensemble de tags : catégorie du produit, localisation, origine, etc.<br>
* Les ingrédients composant les produits et leurs additifs éventuels.<br>
* Des informations nutritionnelles : quantité en grammes d’un nutriment pour 100 grammes du produit.
<br>
**********


## <font color=#7451eb>CONTENU DU NOTEBOOK</font> :
<br>

&#9989; **Chapitre 1** : **Inspection du dataset**.
* 1.1 Chargement et exploration du datase.<br>
* 1.2 Traitement des colonnes avec un mixte data type.<br>
* 1.3 Création d'une fonction pour convertir les colonnes vers le bon dtype.<br>
* 1.4 Traitement des colonnes avec un datatype en timestamp et/ou iso8601.<br>
* 1.5 Création d'une pour le formatage des colonnes en timestamp et iso8601.<br>
* 1.6 Exploration du dataset (shape, dtypes, nan values, doublons ...etc).<br>
* 1.7 Traitement des fake NaN (valeurs en string).<br>
* 1.8 Création d'une fonction pour l'identification et formatage des fake NaN.<br>
* 1.9 Conclusion<br>
**********
&#9989; **Chapitre 2** : **Idée d'application**
* 2.1 Concept de l'application<br>
**********
&#9989; **Chapitre 3** : **Nettoyage du dataset**.<br>
* 3.1 Supprimer les doublons identifiés sur la colonne code.<br>
* 3.2 Conversion l'unité de la colonne "sodium_100g" de `g` en `mg`.<br>
* 3.3 Nettoyage des colonnes `countries` et `manufacturing_places`.<br>
* 3.4 Creation de la formule de calcul du Nutri-Score & du Nutri-Grade.<br>
* 3.5 Imputation des valeurs manquantes sur les variables numériques (sauf le nutri-score  et le nutri-grade).<br>
* 3.6 Création d'une fonction pour **AUTOMATISER** l'imputation des valeurs manquantes avec `KNNImputer`.<br>
**********
&#9989; **Chapitre 4** : **Automatiser le nettoyage du dataset**
* 4.1 Création d'une fonction pour **AUTOMATISER tous les traitements** pour nettoyer le dataset.
* 4.2 Test de la fonction de  nettoyage du dataset.
* 4.3 Exploration du dataset cleané.
**********

*********
# <font color=#7451eb> Chapitre 1 : Inspection du dataset</font>
*********

In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
pd.options.display.max_rows = 200
pd.options.display.max_columns = 200
# pd.options.display.max_info_columns = 200
# pd.options.display.max_info_rows = 200
# pd.reset_option("all")

### <img src='./images/logo_OC.png' width=20px align="left"/> &emsp; Chargement et exploration du datase.

In [3]:
%%time
df = pd.read_csv("data/en.openfoodfacts.org.products.csv", sep="\t")

C:\Users\Mohamed-Sidina\Desktop\OpenClassRooms_Data_Science\00_Learning_Training_Env\lib\site-packages\decorator.py:232: DtypeWarning: Columns (0,8,13,22,23,27,28,29,31,38,47,52,55,64,75,77) have mixed types.Specify dtype option on import or set low_memory=False.
  return caller(func, *(extras + args), **kw)


Wall time: 2min 33s


In [4]:
df.head()

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,abbreviated_product_name,generic_name,quantity,packaging,packaging_tags,packaging_text,brands,brands_tags,categories,categories_tags,categories_en,origins,origins_tags,origins_en,manufacturing_places,manufacturing_places_tags,labels,labels_tags,labels_en,emb_codes,emb_codes_tags,first_packaging_code_geo,cities,cities_tags,purchase_places,stores,countries,countries_tags,countries_en,ingredients_text,allergens,allergens_en,traces,traces_tags,traces_en,serving_size,serving_quantity,no_nutriments,additives_n,additives,additives_tags,additives_en,ingredients_from_palm_oil_n,ingredients_from_palm_oil,ingredients_from_palm_oil_tags,ingredients_that_may_be_from_palm_oil_n,ingredients_that_may_be_from_palm_oil,ingredients_that_may_be_from_palm_oil_tags,nutriscore_score,nutriscore_grade,nova_group,pnns_groups_1,pnns_groups_2,states,states_tags,states_en,brand_owner,ecoscore_score_fr,ecoscore_grade_fr,main_category,main_category_en,image_url,image_small_url,image_ingredients_url,image_ingredients_small_url,image_nutrition_url,image_nutrition_small_url,energy-kj_100g,energy-kcal_100g,energy_100g,energy-from-fat_100g,fat_100g,saturated-fat_100g,-butyric-acid_100g,-caproic-acid_100g,-caprylic-acid_100g,-capric-acid_100g,-lauric-acid_100g,-myristic-acid_100g,-palmitic-acid_100g,-stearic-acid_100g,-arachidic-acid_100g,-behenic-acid_100g,-lignoceric-acid_100g,-cerotic-acid_100g,-montanic-acid_100g,-melissic-acid_100g,monounsaturated-fat_100g,polyunsaturated-fat_100g,omega-3-fat_100g,-alpha-linolenic-acid_100g,-eicosapentaenoic-acid_100g,-docosahexaenoic-acid_100g,omega-6-fat_100g,-linoleic-acid_100g,-arachidonic-acid_100g,-gamma-linolenic-acid_100g,-dihomo-gamma-linolenic-acid_100g,omega-9-fat_100g,-oleic-acid_100g,-elaidic-acid_100g,-gondoic-acid_100g,-mead-acid_100g,-erucic-acid_100g,-nervonic-acid_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,-sucrose_100g,-glucose_100g,-fructose_100g,-lactose_100g,-maltose_100g,-maltodextrins_100g,starch_100g,polyols_100g,fiber_100g,-soluble-fiber_100g,-insoluble-fiber_100g,proteins_100g,casein_100g,serum-proteins_100g,nucleotides_100g,salt_100g,sodium_100g,alcohol_100g,vitamin-a_100g,beta-carotene_100g,vitamin-d_100g,vitamin-e_100g,vitamin-k_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-pp_100g,vitamin-b6_100g,vitamin-b9_100g,folates_100g,vitamin-b12_100g,biotin_100g,pantothenic-acid_100g,silica_100g,bicarbonate_100g,potassium_100g,chloride_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,fluoride_100g,selenium_100g,chromium_100g,molybdenum_100g,iodine_100g,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,fruits-vegetables-nuts-dried_100g,fruits-vegetables-nuts-estimate_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,carbon-footprint-from-meat-or-fish_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g,choline_100g,phylloquinone_100g,beta-glucan_100g,inositol_100g,carnitine_100g
0,00000000000000225,http://world-en.openfoodfacts.org/product/0000...,nutrinet-sante,1623855208,2021-06-16T14:53:28Z,1623855209,2021-06-16T14:53:29Z,jeunes pousses,NaN,NaN,NaN,NaN,NaN,NaN,endives,endives,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en:france,en:france,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unknown,unknown,"en:to-be-completed, en:nutrition-facts-to-be-c...","en:to-be-completed,en:nutrition-facts-to-be-co...","To be completed,Nutrition facts to be complete...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [5]:
df.shape

(1847483, 186)

### <img src='./images/logo_OC.png' width=20px align="left"/> &emsp; Traitement des colonnes avec un mixte data type.

In [6]:
df.iloc[:, [0, 8, 13, 22, 23, 27, 28, 29, 31, 38, 47, 52, 55, 64, 75, 77]]

,code,abbreviated_product_name,packaging_text,manufacturing_places,manufacturing_places_tags,emb_codes,emb_codes_tags,first_packaging_code_geo,cities_tags,allergens,additives,ingredients_from_palm_oil_tags,ingredients_that_may_be_from_palm_oil_tags,brand_owner,energy-kj_100g,energy_100g
0,00000000000000225,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0000000000017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1569.0
2,0000000000031,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,000000000003327986,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,685.8,685.8
4,000000000004622327,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3661.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1847478,9999999910128,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1847479,999999999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1847480,9999999990397,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0
1847481,9999999999994,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


<img src='./images/logo_OC.png' width=20px align="left"/> &emsp; Afficher les mixtes data types.

In [7]:
mixed_types_columns_index = [
    0,
    8,
    13,
    22,
    23,
    27,
    28,
    29,
    31,
    38,
    47,
    52,
    55,
    64,
    75,
    77,
]

In [8]:
for c in mixed_types_columns_index:
    print("colonne :", df.columns[c])
    print("types de données", set(df.iloc[:, c].apply(type)))
    print("-" * 50)

colonne : code
types de données {<class 'str'>, <class 'int'>}
--------------------------------------------------
colonne : abbreviated_product_name
types de données {<class 'str'>, <class 'float'>}
--------------------------------------------------
colonne : packaging_text
types de données {<class 'str'>, <class 'float'>}
--------------------------------------------------
colonne : manufacturing_places
types de données {<class 'str'>, <class 'float'>}
--------------------------------------------------
colonne : manufacturing_places_tags
types de données {<class 'str'>, <class 'float'>}
--------------------------------------------------
colonne : emb_codes
types de données {<class 'str'>, <class 'float'>}
--------------------------------------------------
colonne : emb_codes_tags
types de données {<class 'str'>, <class 'float'>}
--------------------------------------------------
colonne : first_packaging_code_geo
types de données {<class 'str'>, <class 'float'>}
-----------------------

 <img src='./images/logo_OC.png' width=20px align="left"/> &emsp; Afficher les valeurs uniques par colonne, afin de choisir le meilleur data type.


In [9]:
for c in mixed_types_columns_index:
    print("colonne:", df.iloc[:, [c]].drop_duplicates())
    print("-" * 50)

colonne:                        code
0         00000000000000225
1             0000000000017
2             0000000000031
3        000000000003327986
4        000000000004622327
...                     ...
1847478       9999999910128
1847479           999999999
1847480       9999999990397
1847481       9999999999994
1847482     999999999999999

[1847149 rows x 1 columns]
--------------------------------------------------
colonne:           abbreviated_product_name
0                              NaN
651702       GLORIA LCNS 7.5% 410g
651703     GLORIA LCNS 7.5% 3x410g
651704         GLORIA LCNS 4% 410g
651705       GLORIA LCNS 4% 3x410g
...                            ...
1844745        Dove original 100ml
1844746  Dove beauty finish 100 ml
1844761         Axe bs black 100ml
1844803    Dove zero original 50ml
1844815   Msv bs 100ml fleur lotus

[6464 rows x 1 columns]
--------------------------------------------------
colonne:                                             packaging_text
0  

colonne:                          brand_owner
0                                NaN
777                           LAGG'S
790            BIG SKY BREAD COMPANY
794      Heartlight Corp. Of America
810              TODAY'S TEMPTATIONS
...                              ...
1843896  HSC INTERNATIONAL PTE. LTD.
1843966                        YEO'S
1844161         Lee Biscuits Pte Ltd
1845004             MONARI FERERZONI
1846098                  CASABE PAUL

[25571 rows x 1 columns]
--------------------------------------------------
colonne:         energy-kj_100g
0                  NaN
3                685.8
5                936.0
173             1513.0
289             2179.0
...                ...
1838600           82.1
1844229          95.45
1845696          836.8
1846377           9.88
1846840         1637.3

[5055 rows x 1 columns]
--------------------------------------------------
colonne:         energy_100g
0               NaN
1            1569.0
3             685.8
4            3661.0


|Colonne|Types de données actuels| Type de données choisi | Pourquoi ?
|:-|:-|:-|:-|
|Code|`int` et `str`|`str`| Le code est un identifiant unique de chaque produit ==> ne pas être utilisé pour faire des calculs mathématiques|
|abbreviated_product_name|`float` et `str`|`str`|Liste des abbreviations des noms des produits|
|packaging_text|`float` et `str`|`str`|Descriptif de l'emballage de chaque produit|
|manufacturing_places|`float` et `str`|`str`|Liste des pays fabricants des produits|
|manufacturing_places_tags|`float` et `str`|`str`|Liste des pays fabricants des produits|
|emb_codes|`float` et `str`|`str`|Liste des codes de traçabilité pour les produits|
|emb_codes_tags|`float` et `str`|`str`|Liste des codes de traçabilité pour les produits|
|first_packaging_code_geo|`float` et `str`|`str`|combinaison des coordonnées géographiques(la longitude et l'altitude)|
|cities_tags|`float` et `str`|`str`|Nom des villes|
|allergens|`float` et `str`|`str`|liste des noms des allergènes|
|additives|`float` et `str`|`str`|les noms des additifs|
|ingredients_from_palm_oil_tags|`float` et `str`|`str`|Présence (Oui ou Non) d'ingrédients issus de l'huile de palme| 
|ingredients_that_may_be_from_palm_oil_tags|`float` et `str`|`str`|Liste des ingrédients potentiellement dérivés de l'huile de palme|
|brand_owner|`float` et `str`|`str`|Le nom du propriétaire de la marque est un texte donc du `str` foncement|
|energy-kj_100g|`float` et `str`|`float`|Contient des values numériques avec parfois des virgules|
|energy_100g|`float` et `str`|`float`|Contient des values numériques avec parfois des virgules |


### <img src='./images/logo_OC.png' width=20px align="left"/> &emsp; Création d'une fonction pour convertir les colonnes vers le bon dtype

In [10]:
def mixte_type_conversion(dataframe):

    """
    Cette fonction sera charge de convertir les colonnes avec un mixte data type vers celui défini dans le tableau précédemment.
    """
    liste_colonnes_float = [
        "energy-kj_100g",
        "energy_100g",
        "additives_n",
        "energy_100g",
        "fat_100g",
        "saturated-fat_100g",
        "sugars_100g",
        "proteins_100g",
        "salt_100g",
        "fiber_100g",
        "fruits-vegetables-nuts_100g",
        "nutriscore_score",
    ]

    liste_colonnes_str = [
        "code",
        "abbreviated_product_name",
        "packaging_text",
        "manufacturing_places",
        "manufacturing_places_tags",
        "emb_codes",
        "emb_codes_tags",
        "first_packaging_code_geo",
        "cities_tags",
        "allergens",
        "additives",
        "ingredients_that_may_be_from_palm_oil_tags",
        "brand_owner",
        "ingredients_from_palm_oil_tags",
        "url",
        "product_name",
        "pnns_groups_1",
        "pnns_groups_2",
        "countries",
        "nutriscore_grade",
    ]

    #     liste_colonnes_bool = ["ingredients_from_palm_oil_tags"]

    for f in liste_colonnes_float:
        dataframe.loc[:, f] = dataframe.loc[:, f].astype(float)
    else:
        for s in liste_colonnes_str:
            dataframe.loc[:, s] = dataframe.loc[:, s].astype(str)

    #     for (s, b) in zip(liste_colonnes_str, dataframe.columns.tolist()) :
    #         if (s == b)  :
    #             dataframe[s] = dataframe[s].astype(str)

    #         else :
    #             for (f, a) in zip(liste_colonnes_float, dataframe.columns.tolist()):
    #                 if (f == a)  & (f != s):
    #                     dataframe[f] = dataframe[f].astype(float)

    return print("La conversion des Dtypes a été correctement réalisée")

In [11]:
mixte_type_conversion(df)

La conversion des Dtypes a été correctement réalisée


In [12]:
df.dtypes.sort_values()

created_t                                       int64
last_modified_t                                 int64
-cerotic-acid_100g                            float64
-fructose_100g                                float64
-lactose_100g                                 float64
-maltose_100g                                 float64
-maltodextrins_100g                           float64
starch_100g                                   float64
polyols_100g                                  float64
fiber_100g                                    float64
-soluble-fiber_100g                           float64
-insoluble-fiber_100g                         float64
casein_100g                                   float64
-glucose_100g                                 float64
serum-proteins_100g                           float64
nucleotides_100g                              float64
salt_100g                                     float64
sodium_100g                                   float64
alcohol_100g                

### <img src='./images/logo_OC.png' width=20px align="left"/> &emsp; Traitement des colonnes avec un datatype en timestamp et/ou iso8601 .

<img src='./images/logo_OC.png' width=20px align="left"/> &emsp; Visualiser les colonnes avec un datatype en timestamp et iso8601 .

In [13]:
# - fields that end with _t are dates in the UNIX timestamp format (number of seconds since Jan 1st 1970)
# - fields that end with _datetime are dates in the iso8601 format: yyyy-mm-ddThh:mn:ssZ
df[
    df.columns[df.columns.str.endswith("_t") + df.columns.str.endswith("_datetime")]
].head()

,created_t,created_datetime,last_modified_t,last_modified_datetime
0,1623855208,2021-06-16T14:53:28Z,1623855209,2021-06-16T14:53:29Z
1,1529059080,2018-06-15T10:38:00Z,1561463718,2019-06-25T11:55:18Z
2,1539464774,2018-10-13T21:06:14Z,1539464817,2018-10-13T21:06:57Z
3,1574175736,2019-11-19T15:02:16Z,1624390765,2021-06-22T19:39:25Z
4,1619501895,2021-04-27T05:38:15Z,1619501897,2021-04-27T05:38:17Z


### <img src='./images/logo_OC.png' width=20px align="left"/> &emsp; Création d'une pour le formatage des colonnes en timestamp et iso8601 .

In [14]:
def datetime_conversion(dataframe):
    """
    Cette fonction sera charge de convertir les colonnes avec un data type UNIX timestamp ou iso8601 format vers un datetime standard.
    """
    liste_colonnes_data = [
        "created_t",
        "created_datetime",
        "last_modified_t",
        "last_modified_datetime",
    ]
    for d in liste_colonnes_data:
        if d[-2:] == "_t":
            dataframe.loc[:, d] = pd.to_datetime(dataframe.loc[:, d], unit="s")

        else:
            dataframe.loc[:, d] = pd.to_datetime(
                dataframe.loc[:, d], infer_datetime_format=True
            )

    return dataframe[
        dataframe.columns[
            dataframe.columns.str.endswith("_t")
            + dataframe.columns.str.endswith("_datetime")
        ]
    ].head()

In [15]:
datetime_conversion(df)

,created_t,created_datetime,last_modified_t,last_modified_datetime
0,2021-06-16 14:53:28,2021-06-16 14:53:28,2021-06-16 14:53:29,2021-06-16 14:53:29
1,2018-06-15 10:38:00,2018-06-15 10:38:00,2019-06-25 11:55:18,2019-06-25 11:55:18
2,2018-10-13 21:06:14,2018-10-13 21:06:14,2018-10-13 21:06:57,2018-10-13 21:06:57
3,2019-11-19 15:02:16,2019-11-19 15:02:16,2021-06-22 19:39:25,2021-06-22 19:39:25
4,2021-04-27 05:38:15,2021-04-27 05:38:15,2021-04-27 05:38:17,2021-04-27 05:38:17


In [16]:
df.head(2)

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,abbreviated_product_name,generic_name,quantity,packaging,packaging_tags,packaging_text,brands,brands_tags,categories,categories_tags,categories_en,origins,origins_tags,origins_en,manufacturing_places,manufacturing_places_tags,labels,labels_tags,labels_en,emb_codes,emb_codes_tags,first_packaging_code_geo,cities,cities_tags,purchase_places,stores,countries,countries_tags,countries_en,ingredients_text,allergens,allergens_en,traces,traces_tags,traces_en,serving_size,serving_quantity,no_nutriments,additives_n,additives,additives_tags,additives_en,ingredients_from_palm_oil_n,ingredients_from_palm_oil,ingredients_from_palm_oil_tags,ingredients_that_may_be_from_palm_oil_n,ingredients_that_may_be_from_palm_oil,ingredients_that_may_be_from_palm_oil_tags,nutriscore_score,nutriscore_grade,nova_group,pnns_groups_1,pnns_groups_2,states,states_tags,states_en,brand_owner,ecoscore_score_fr,ecoscore_grade_fr,main_category,main_category_en,image_url,image_small_url,image_ingredients_url,image_ingredients_small_url,image_nutrition_url,image_nutrition_small_url,energy-kj_100g,energy-kcal_100g,energy_100g,energy-from-fat_100g,fat_100g,saturated-fat_100g,-butyric-acid_100g,-caproic-acid_100g,-caprylic-acid_100g,-capric-acid_100g,-lauric-acid_100g,-myristic-acid_100g,-palmitic-acid_100g,-stearic-acid_100g,-arachidic-acid_100g,-behenic-acid_100g,-lignoceric-acid_100g,-cerotic-acid_100g,-montanic-acid_100g,-melissic-acid_100g,monounsaturated-fat_100g,polyunsaturated-fat_100g,omega-3-fat_100g,-alpha-linolenic-acid_100g,-eicosapentaenoic-acid_100g,-docosahexaenoic-acid_100g,omega-6-fat_100g,-linoleic-acid_100g,-arachidonic-acid_100g,-gamma-linolenic-acid_100g,-dihomo-gamma-linolenic-acid_100g,omega-9-fat_100g,-oleic-acid_100g,-elaidic-acid_100g,-gondoic-acid_100g,-mead-acid_100g,-erucic-acid_100g,-nervonic-acid_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,-sucrose_100g,-glucose_100g,-fructose_100g,-lactose_100g,-maltose_100g,-maltodextrins_100g,starch_100g,polyols_100g,fiber_100g,-soluble-fiber_100g,-insoluble-fiber_100g,proteins_100g,casein_100g,serum-proteins_100g,nucleotides_100g,salt_100g,sodium_100g,alcohol_100g,vitamin-a_100g,beta-carotene_100g,vitamin-d_100g,vitamin-e_100g,vitamin-k_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-pp_100g,vitamin-b6_100g,vitamin-b9_100g,folates_100g,vitamin-b12_100g,biotin_100g,pantothenic-acid_100g,silica_100g,bicarbonate_100g,potassium_100g,chloride_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,fluoride_100g,selenium_100g,chromium_100g,molybdenum_100g,iodine_100g,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,fruits-vegetables-nuts-dried_100g,fruits-vegetables-nuts-estimate_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,carbon-footprint-from-meat-or-fish_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g,choline_100g,phylloquinone_100g,beta-glucan_100g,inositol_100g,carnitine_100g
0,00000000000000225,http://world-en.openfoodfacts.org/product/0000...,nutrinet-sante,2021-06-16 14:53:28,2021-06-16 14:53:28,2021-06-16 14:53:29,2021-06-16 14:53:29,jeunes pousses,nan,NaN,NaN,NaN,NaN,nan,endives,endives,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,NaN,NaN,NaN,nan,nan,nan,NaN,nan,NaN,NaN,en:france,en:france,France,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,NaN,NaN,NaN,nan,NaN,NaN,nan,NaN,nan,NaN,unknown,unknown,"en:to-be-completed, en:nutrition-facts-to-be-c...","en:to-be-completed,en:nutrition-facts-to-be-co...","To be completed,Nutrition facts to be complete...",nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [17]:
sum(df["created_t"] == df["created_datetime"]) - len(df["created_t"])

0

In [18]:
sum(df["last_modified_t"] == df["last_modified_datetime"]) - len(df["created_t"])

0

* Les colonnes **created_t** et **created_datetime** sont identiques.
* Les colonnes **last_modified_t** et **last_modified_datetime** sont identiques

### <img src='./images/logo_OC.png' width=20px align="left"/> &emsp; Exploration du dataset.

### <img src='./images/logo_OC.png' width=20px align="left"/> &emsp; Affichons l'ensemble des noms des variables/colonnes.

In [19]:
df.columns.sort_values().to_list()

['-alpha-linolenic-acid_100g',
 '-arachidic-acid_100g',
 '-arachidonic-acid_100g',
 '-behenic-acid_100g',
 '-butyric-acid_100g',
 '-capric-acid_100g',
 '-caproic-acid_100g',
 '-caprylic-acid_100g',
 '-cerotic-acid_100g',
 '-dihomo-gamma-linolenic-acid_100g',
 '-docosahexaenoic-acid_100g',
 '-eicosapentaenoic-acid_100g',
 '-elaidic-acid_100g',
 '-erucic-acid_100g',
 '-fructose_100g',
 '-gamma-linolenic-acid_100g',
 '-glucose_100g',
 '-gondoic-acid_100g',
 '-insoluble-fiber_100g',
 '-lactose_100g',
 '-lauric-acid_100g',
 '-lignoceric-acid_100g',
 '-linoleic-acid_100g',
 '-maltodextrins_100g',
 '-maltose_100g',
 '-mead-acid_100g',
 '-melissic-acid_100g',
 '-montanic-acid_100g',
 '-myristic-acid_100g',
 '-nervonic-acid_100g',
 '-oleic-acid_100g',
 '-palmitic-acid_100g',
 '-soluble-fiber_100g',
 '-stearic-acid_100g',
 '-sucrose_100g',
 'abbreviated_product_name',
 'additives',
 'additives_en',
 'additives_n',
 'additives_tags',
 'alcohol_100g',
 'allergens',
 'allergens_en',
 'beta-carotene

 <img src='./images/logo_OC.png' width=20px align="left"/> &emsp; Visualiser les types de donées pour chaque colonnes.

In [20]:
df.dtypes.sort_values()

-cerotic-acid_100g                                   float64
-glucose_100g                                        float64
-fructose_100g                                       float64
-lactose_100g                                        float64
-maltose_100g                                        float64
-maltodextrins_100g                                  float64
starch_100g                                          float64
polyols_100g                                         float64
fiber_100g                                           float64
-soluble-fiber_100g                                  float64
-insoluble-fiber_100g                                float64
proteins_100g                                        float64
casein_100g                                          float64
serum-proteins_100g                                  float64
nucleotides_100g                                     float64
salt_100g                                            float64
sodium_100g             

In [21]:
df.dtypes.value_counts()

float64           123
object             59
datetime64[ns]      4
dtype: int64

 <img src='./images/logo_OC.png' width=20px align="left"/> &emsp; Comptage du nombre de colonnes dont les titres commencent avec le caractère spécial `-`.

In [22]:
df.columns.str.startswith("-").sum()

35

### <img src='./images/logo_OC.png' width=20px align="left"/>&emsp;  Traitement des faux NaN (valeurs en string) sur les colonnes de type `object`

 <img src='./images/logo_OC.png' width=20px align="left"/>&emsp;  Comptage du nombre des faux NaN.

In [23]:
np.sum(df == "nan").sum()

24114079

In [24]:
def correct_fake_NaN(x):
    """
    Cette fonction corrige les Fake NaN values.
    """
    if x in ["Nan", "nan", "naN", "nAn", "NaN", "NAN"]:
        return float("nan")
    return x

In [25]:
df = df.applymap(correct_fake_NaN)

In [26]:
np.sum(df == "nan").sum()

0

<img src='./images/logo_OC.png' width=20px align="left"/> &emsp; Vérifions le % de valeurs manquantes par ligne.

In [27]:
df.isna().mean().round(4).sort_values(ascending=False) * 100

-montanic-acid_100g                           100.00
-palmitic-acid_100g                           100.00
-dihomo-gamma-linolenic-acid_100g             100.00
-gamma-linolenic-acid_100g                    100.00
-melissic-acid_100g                           100.00
-cerotic-acid_100g                            100.00
-lignoceric-acid_100g                         100.00
-behenic-acid_100g                            100.00
-stearic-acid_100g                            100.00
-myristic-acid_100g                           100.00
-elaidic-acid_100g                            100.00
-lauric-acid_100g                             100.00
-capric-acid_100g                             100.00
-caprylic-acid_100g                           100.00
-caproic-acid_100g                            100.00
-butyric-acid_100g                            100.00
ingredients_that_may_be_from_palm_oil         100.00
ingredients_from_palm_oil                     100.00
-oleic-acid_100g                              

<img src='./images/logo_OC.png' width=20px align="left"/> &emsp; Quel est le nombre de colonnes qui ont un % de valeurs manquantes  >=  90%.

In [28]:
sum(df.isna().mean().round(4).sort_values(ascending=False) * 100 >= 90.00)

118

 <img src='./images/logo_OC.png' width=20px align="left"/> &emsp; Vérifions s'il existe des valeurs en double 
pour la variable `code` qui est un identifiant unique de chaque produit.

In [29]:
df.duplicated("code").sum()

839

# <font> Chapitre 1 : Conclusion </font>

* Le dataset est constitué de **1 847 483 lignes** et **186 colonnes**.
* Le dataset contient : 
    * **123** colonnes avec des données de type `float`.
    *  **58** colonnes avec des données de type `object`.
    * **4** colonnes avec des données de type `datetime`. 
    * et **1** colonne avec des données de type `bool`. 
* Sur les **186** colonnes présentes dans le dataset **117** contiennent **`plus de 90%`** de valeurs manquantes.
* La colonne `code` contient **839** `doublons` ==> à supprimer dans la suite de l'analyse.
* **35 colonnes** ont un nom qui commence par un caractère special `-` 
* **16** colonnes contiennent des types de données mixtes.
* Plusieurs **colonnes** sont des **doublons** (comme additives, additives_tags, packaging_tags
packaging_text..etc)
* Les colonnes **created_t** et **created_datetime** sont identiques.
* Les colonnes **last_modified_t** et **last_modified_datetime** sont identiques.
<br>
<br>
* **NB**:  Dans la suite de l'analyse, nous allons garder uniquement les colonnes en lien avec l'idée de l'application.

*********
# <font color=#7451eb> Chapitre 2 : Idée d'application</font>
*********


### <img src='./images/logo_OC.png' width=20px align="left"/> &emsp; Concept de l'application.

L'objectif de cette application est d'**aider les consommateurs** à **changer** de **comportement** en faveur d'une **nutrition **plus saine**, en même temps participer au **développement de l'économie locale** en achetant plus de produits **Made in France**.
<br>

**Explications du principe de fonctionnement de l'App ?**
<br>
* Etape 1 - Le consommateur **scanne** son **produit**.<br>
* Etape 2 - L'application retourne **une image avec le nutri-grade**.<br>
* Etape 3 - L'application affiche également le **logo, Made in France** si le produit a été fabriqué en France.
* Etape 4 - Si le produit est  **fabriqué à l'étranger** l'application doit proposer un **produit similaire** avec un **score similaire, voire meilleur** et qui est **Made in France**.
<br>


Selon [Sante Publique France](https://www.santepubliquefrance.fr/determinants-de-sante/nutrition-et-activite-physique/articles/nutri-score)
<br> l'indicateur nutri-score, se calcul en prenant en compte pour **100 gr** ou **100 mL** de **produit**, **la teneur** :

* en **nutriments et aliments à favoriser** (fibres, protéines, fruits, légumes, légumineuses, fruits à coques, huile de colza, de noix et d’olive),
* et en **nutriments à limiter** (énergie, acides gras saturés, sucres, sel).

**Dans la suite du notebook, nous allons garder uniquement les colonnes en lien avec l'idée de l'application**                
                     
                  

*********
# <font color=#7451eb> Chapitre 3 : Nettoyage du dataset</font>
*********

### <img src='./images/logo_OC.png' width=20px align="left"/> &emsp; Suppression les doublons identifiés sur la colonne `code`</font>


In [46]:
app_idea_Columns = [
    "code",
    "url",
    "product_name",
    "pnns_groups_1",
    "pnns_groups_2",
    "additives_n",
    "countries",
    "manufacturing_places",
    "energy_100g",
    "fat_100g",
    "saturated-fat_100g",
    "sugars_100g",
    "proteins_100g",
    "salt_100g",
    "fiber_100g",
    "fruits-vegetables-nuts_100g",
    "nutriscore_score",
    "nutriscore_grade",
]

In [47]:
df_app = df[app_idea_Columns].copy()

In [48]:
df_app = df_app.drop_duplicates(subset=["code"])

### <img src='./images/logo_OC.png' width=20px align="left"/> &emsp; Conversion l'unité de la colonne "sodium_100g" de `g` en `mg`.
Cette conversion est nécessaire pour le bon fonctionnement de la formule de calcul.
Pour réaliser cette conversion, nous allons utiliser la colonne `salt_100g` comme recommandé dans l'outil de calcul de Santé publique France.

In [49]:
df_app["sodium_100mg"] = (df_app["salt_100g"].div(2.5)) * 1000

In [50]:
df_app.isna().mean().sort_values(ascending=False) * 100

fruits-vegetables-nuts_100g    99.524164
manufacturing_places           93.686764
fiber_100g                     74.172715
nutriscore_grade               63.126244
nutriscore_score               63.126244
additives_n                    62.062368
sodium_100mg                   24.464921
salt_100g                      24.464921
saturated-fat_100g             23.465216
sugars_100g                    22.040252
fat_100g                       21.096974
proteins_100g                  21.034644
energy_100g                    20.660560
product_name                    4.275702
countries                       0.303740
pnns_groups_1                   0.048412
pnns_groups_2                   0.044784
url                             0.000000
code                            0.000000
dtype: float64

In [51]:
df_app.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1846644 entries, 0 to 1847482
Data columns (total 19 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   code                         object 
 1   url                          object 
 2   product_name                 object 
 3   pnns_groups_1                object 
 4   pnns_groups_2                object 
 5   additives_n                  float64
 6   countries                    object 
 7   manufacturing_places         object 
 8   energy_100g                  float64
 9   fat_100g                     float64
 10  saturated-fat_100g           float64
 11  sugars_100g                  float64
 12  proteins_100g                float64
 13  salt_100g                    float64
 14  fiber_100g                   float64
 15  fruits-vegetables-nuts_100g  float64
 16  nutriscore_score             float64
 17  nutriscore_grade             object 
 18  sodium_100mg                 float64
dtype

### <img src='./images/logo_OC.png' width=20px align="left"/> &emsp; Nettoyage des colonnes `countries` et `manufacturing_places`

<img src='./images/logo_OC.png' width=20px align="left"/> &emsp; Colonne : `countries`, supression des valeurs manquantes.

In [52]:
df_app.dropna(subset=["countries"], inplace=True)

In [53]:
df_app["countries"].isna().sum()

0

 <img src='./images/logo_OC.png' width=20px align="left"/> &emsp; Afficher les valeurs uniques de la colonne : `countries`.

In [54]:
pd.options.display.max_rows = None

In [55]:
df_app["countries"].drop_duplicates()

0                                                  en:france
1                                                     France
3                                                    Espagne
4                                                      en:es
6                                                  en:France
12                                                     en:fr
57                                             United States
78                                                   Irlande
84                                                     en:FR
110                                                    en:be
173                                           United Kingdom
188                                    États-Unis, en:france
237                                    France, United States
272                                        France,États-Unis
331                                                    Spain
332                                           en:Ivory Coast
337                     

 <img src='./images/logo_OC.png' width=20px align="left"/> &emsp; Afficher les valeurs uniques de la colonne : `countries`
qui contiennent `fr` pour La France.


In [40]:
df_app[df_app["countries"].str.contains("fr", case=False, na=False)][
    "countries"
].drop_duplicates()

0                                                  en:france
1                                                     France
6                                                  en:France
12                                                     en:fr
84                                                     en:FR
188                                    États-Unis, en:france
237                                    France, United States
272                                        France,États-Unis
401                                          France,Belgique
413                                       France, États-Unis
608                                  Frankreich, Deutschland
609                               France,Germany,Netherlands
649                                       France,Switzerland
653                                        Frankreich, en:de
654                                            France, en:de
980                                          Belgique,France
1285                    

 <img src='./images/logo_OC.png' width=20px align="left"/> &emsp; Formater le contenu de la colonne : `countries`
et garder uniquement `La France`

In [41]:
Produits_Vendus_En_France = [
    "en:france",
    "France",
    "france",
    "FRANCE",
    "en:France",
    "en:fr",
    "en:FR",
    "Frankreich",
    "Francia",
    "La Réunion",
    "Réunion",
    "French Polynesia",
    "french-polynesia",
    "New Caledonia",
    "Polynésie française",
    "Guadeloupe",
    "Frankrijk",
    "Martinique",
    "martinique",
    "França",
    "new-caledonia",
    "Nouvelle-Calédonie",
    "Monaco",
    "Frantsa",
    "Fransa",
    "Francia",
    "Mayotte",
    "en:Mayotte",
]

In [42]:
df_app.loc[
    df_app["countries"].isin(Produits_Vendus_En_France), ["countries"]
] = "France"

In [43]:
df_app = df_app.drop(df_app[df_app["countries"] != "France"].index)

In [44]:
df_app["countries"].unique()

array(['France'], dtype=object)

<img src='./images/logo_OC.png' width=20px align="left"/> &emsp; Colonne : `manufacturing_places`, supression des valeurs manquantes.


In [56]:
df_app.dropna(subset=["manufacturing_places"], inplace=True)

In [57]:
df_app["manufacturing_places"].isna().sum()

0

 <img src='./images/logo_OC.png' width=20px align="left"/> &emsp; Afficher les valeurs uniques de la colonne : `manufacturing_places`.


In [59]:
df_app["manufacturing_places"].drop_duplicates()

63                                             bénivay-ollon
164                                                   France
337                                          Brossard Québec
369                                           United Kingdom
394                                          Brossard,Québec
401                                                   Italie
413                                               États-Unis
417                                         France,Avranches
428                                               Etats-unis
494                                          Brossars,Québec
496                                                 Thailand
540                                                  Belgien
585                                                   net wt
738                                               Österreich
747        Las Ventas de Retamosa,Toledo (provincia),Cast...
766                                      Saint Yrieix,France
807                     

 <img src='./images/logo_OC.png' width=20px align="left"/> &emsp; Afficher les valeurs uniques de la colonne : `manufacturing_places`
qui contiennent `fr` pour La France.

In [60]:
df_app[df_app["manufacturing_places"].str.contains("fr", case=False, na=False)][
    "manufacturing_places"
].drop_duplicates()

164                                                   France
417                                         France,Avranches
766                                      Saint Yrieix,France
920                       France,Limousin,87500,Saint-Yrieux
974                                             France,87500
996                                             87500,France
1490                                         Périgord France
2650                                                  france
2848       Argentina - Español,Aruba - Español,Bolivia - ...
4503                                     la halle aux fruits
5335                                         Limousin France
13834                               SAN FRANCISCO CALIFORNIA
23551        Gerber Products Company  Fremont   MI 49413 USA
42727      Madeleines BIJOU,Les Lacs 87500 St-Yrieix la P...
45188                                    07000 Privas,France
49675                Crèmerie Soignon,Poitou-Charente,France
67432                   

 <img src='./images/logo_OC.png' width=20px align="left"/> &emsp; Formater le contenu de la colonne : `manufacturing_places`
et garder uniquement 2 valeurs, `Made in France` et `Not Made in France`

In [61]:
df_app.loc[
    df_app["manufacturing_places"].str.contains("fr", case=False, na=False),
    ["manufacturing_places"],
] = "Made in France"

In [62]:
df_app.loc[
    ~df_app["manufacturing_places"].str.contains("fr", case=False, na=False),
    ["manufacturing_places"],
] = "Not Made in France"

In [63]:
df_app["manufacturing_places"].unique()

array(['Not Made in France', 'Made in France'], dtype=object)

In [64]:
df_app.head()

,code,url,product_name,pnns_groups_1,pnns_groups_2,additives_n,countries,manufacturing_places,energy_100g,fat_100g,saturated-fat_100g,sugars_100g,proteins_100g,salt_100g,fiber_100g,fruits-vegetables-nuts_100g,nutriscore_score,nutriscore_grade,sodium_100mg
63,0000000005,http://world-en.openfoodfacts.org/product/0000...,Nectar d'abricot,Beverages,Sweetened beverages,0.0,en:France,Not Made in France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
164,0000000020114,http://world-en.openfoodfacts.org/product/0000...,Naturablue original,unknown,unknown,1.0,France,Made in France,0.0,0.0,0.0,0.0,0.00,NaN,NaN,NaN,NaN,NaN,NaN
334,0000000274722,http://world-en.openfoodfacts.org/product/0000...,Blanquette de Volaille et son Riz,Composite foods,One-dish meals,2.0,France,Made in France,450.0,2.2,0.9,0.5,6.80,0.700,0.50,NaN,0.0,b,280.0
337,0000000290616,http://world-en.openfoodfacts.org/product/0000...,Salade Cesar,Fruits and vegetables,Vegetables,3.0,Canada,Not Made in France,1210.0,12.0,7.0,0.0,22.00,2.160,2.00,NaN,6.0,c,864.0
352,0000000394710,http://world-en.openfoodfacts.org/product/0000...,Danoises à la cannelle roulées,Sugary snacks,Biscuits and cakes,8.0,Canada,Not Made in France,1520.0,14.4,NaN,28.1,4.79,0.922,2.05,NaN,NaN,NaN,368.8


In [65]:
df_app.dtypes.sort_values()

fat_100g                       float64
nutriscore_score               float64
fruits-vegetables-nuts_100g    float64
fiber_100g                     float64
salt_100g                      float64
proteins_100g                  float64
sugars_100g                    float64
saturated-fat_100g             float64
sodium_100mg                   float64
additives_n                    float64
energy_100g                    float64
nutriscore_grade                object
manufacturing_places            object
countries                       object
pnns_groups_2                   object
pnns_groups_1                   object
product_name                    object
url                             object
code                            object
dtype: object

###  <img src='./images/logo_OC.png' width=20px align="left"/> &emsp; Creation de la formule de calcul du Nutri-Score & du Nutri-Grade.

L'objectif de ce chapitre est de **construire** la **formule** de **calcul** du **nutri-score** et **nutri-grade**, en se basant sur l'**outil** de calcul communiqué par **Santé publique France**, [voici le lien pour accéder à l'outil](https://www.santepubliquefrance.fr/media/files/02-determinants-de-sante/nutrition-et-activite-physique/nutri-score/tableur-calcul-nutri-score) 

Les 2 formules seront par la suite utilisées pour compléter les valeurs manquantes sur les 2 colonnes : `nutri-score` et `nutri-grade`.

<img src='./images/logo_OC.png' width=20px align="left"/> &emsp; Formule de calcul du **score** de la variable `energy_100g`

In [66]:
def pts_kj_func(x):
    """
    Cette fonction calcul l'indicateur pts_kj, qui est un score de la variable `energy_100g`.
    """
    if x <= 335:
        return 0
    elif x <= 670:
        return 1
    elif x <= 1005:
        return 2
    elif x <= 1340:
        return 3
    elif x <= 1675:
        return 4
    elif x <= 2010:
        return 5
    elif x <= 2345:
        return 6
    elif x <= 2680:
        return 7
    elif x <= 3015:
        return 8
    elif x <= 3350:
        return 9
    else:
        return 10
    return x

<img src='./images/logo_OC.png' width=20px align="left"/> &emsp; Formule de calcul du **score** de la variable `energy_100g` pour des produits de la catégorie `Beverages`

In [67]:
def pts_kj_func_Beverages(x):
    """
    Cette fonction calcul l'indicateur pts_kj, qui est un score de la variable `energy_100g`.
    """
    if x <= 0:
        return 0
    elif x <= 30:
        return 1
    elif x <= 60:
        return 2
    elif x <= 90:
        return 3
    elif x <= 120:
        return 4
    elif x <= 150:
        return 5
    elif x <= 180:
        return 6
    elif x <= 210:
        return 7
    elif x <= 240:
        return 8
    elif x <= 270:
        return 9
    else:
        return 10
    return x

<img src='./images/logo_OC.png' width=20px align="left"/> &emsp; Formule de calcul du **score** de la variable `sugars_100g`

In [68]:
def pts_glus_func(x):
    """
    Cette fonction calcul l'indicateur `pts_glus`, qui est un score de la variable `sugars_100g`.
    """
    if x <= 4.5:
        return 0
    elif x <= 9:
        return 1
    elif x <= 13.5:
        return 2
    elif x <= 18:
        return 3
    elif x <= 22.5:
        return 4
    elif x <= 27:
        return 5
    elif x <= 31:
        return 6
    elif x <= 36:
        return 7
    elif x <= 40:
        return 8
    elif x <= 45:
        return 9
    else:
        return 10
    return x

<img src='./images/logo_OC.png' width=20px align="left"/> &emsp; Formule de calcul du **score** de la variable `sugars_100g` pour des produits de la catégorie `Beverages`

In [69]:
def pts_glus_func_Beverages(x):
    """
    Cette fonction calcul l'indicateur `pts_glus`, qui est un score de la variable `sugars_100g`.
    """
    if x <= 0:
        return 0
    elif x <= 1.5:
        return 1
    elif x <= 3:
        return 2
    elif x <= 4.5:
        return 3
    elif x <= 6:
        return 4
    elif x <= 7.5:
        return 5
    elif x <= 9:
        return 6
    elif x <= 10.5:
        return 7
    elif x <= 12:
        return 8
    elif x <= 13.5:
        return 9
    else:
        return 10
    return x

<img src='./images/logo_OC.png' width=20px align="left"/> &emsp; Formule de calcul du **score** de la variable `AGS_Lip_tot`

In [70]:
def pts_ags_func_fat(x):
    """
    Cette fonction calcul l'indicateur `AGS_Lip_tot`, qui est un score des variables `saturated-fat_100g` et `fat-100g`.
    Cette fonction sera appelée uniquement pour les produits de la famille Added Fat.
    """
    if x < 10:
        return 0
    elif x < 16:
        return 1
    elif x < 22:
        return 2
    elif x < 28:
        return 3
    elif x < 34:
        return 4
    elif x < 40:
        return 5
    elif x < 46:
        return 6
    elif x < 52:
        return 7
    elif x < 58:
        return 8
    elif x < 64:
        return 9
    else:
        return 10
    return x

<img src='./images/logo_OC.png' width=20px align="left"/> &emsp; Formule de calcul du **score** de la variable `saturated-fat_100g`.

In [71]:
def pts_ags_func(x):
    """
    Cette fonction calcul l'indicateur `pts_ags_func`, qui est un score de la variable `saturated-fat_100g`.
    Cette fonction sera appelée pour les produits de toute famille, sauf la famille Added Fat.
    """
    if x <= 1:
        return 0
    elif x <= 2:
        return 1
    elif x <= 3:
        return 2
    elif x <= 4:
        return 3
    elif x <= 5:
        return 4
    elif x <= 6:
        return 5
    elif x <= 7:
        return 6
    elif x <= 8:
        return 7
    elif x <= 9:
        return 8
    elif x <= 10:
        return 9
    else:
        return 10
    return x

<img src='./images/logo_OC.png' width=20px align="left"/> &emsp; Formule de calcul du **score** de la variable `sodium_100mg`

In [72]:
def pts_na_func(x):
    """
    Cette fonction calcul l'indicateur `pts_na`, qui est un score de la variable `sodium_100mg`
    """
    if x <= 90:
        return 0
    elif x <= 180:
        return 1
    elif x <= 270:
        return 2
    elif x <= 360:
        return 3
    elif x <= 450:
        return 4
    elif x <= 540:
        return 5
    elif x <= 630:
        return 6
    elif x <= 720:
        return 7
    elif x <= 810:
        return 8
    elif x <= 900:
        return 9
    else:
        return 10
    return x

<img src='./images/logo_OC.png' width=20px align="left"/> &emsp; Formule de calcul du **score** de la variable `proteins_100g`

In [73]:
def pts_prot_func(x):
    """
    Cette fonction calcul l'indicateur `pts_prot`, qui est un score de la variable `proteins_100g`
    """
    if x <= 1.6:
        return 0
    elif x <= 3.2:
        return 1
    elif x <= 4.8:
        return 2
    elif x <= 6.4:
        return 3
    elif x <= 8:
        return 4
    else:
        return 5
    return x

<img src='./images/logo_OC.png' width=20px align="left"/> &emsp; Formule de calcul du  **score** de la variable `fiber_100g`

In [74]:
def pts_fib_func(x):
    """
    Cette fonction calcul l'indicateur `pts_fib`, qui est un score de la variable `fiber_100g`
    """
    if x <= 0.9:
        return 0
    elif x <= 1.9:
        return 1
    elif x <= 2.8:
        return 2
    elif x <= 3.7:
        return 3
    elif x <= 4.7:
        return 4
    else:
        return 5
    return x

<img src='./images/logo_OC.png' width=20px align="left"/> &emsp; Formule de calcul du  **score** de la variable `fruits-vegetables-nuts_100g`

In [75]:
def pts_FLN_func(x):
    """
    Cette fonction calcul l'indicateur `pts_FLN`, qui est un score de la variable `fruits-vegetables-nuts_100g`
    """
    if x <= 40:
        return 0
    elif x <= 60:
        return 1
    elif x <= 80:
        return 2
    else:
        return 5
    return x

<img src='./images/logo_OC.png' width=20px align="left"/> &emsp; Formule de calcul du  **score** de la variable `fruits-vegetables-nuts_100g` pour les produits de la famille `Beverages`

In [76]:
def pts_FLN_func_Beverages(x):
    """
    Cette fonction calcul l'indicateur `pts_FLN`, qui est un score de la variable `fruits-vegetables-nuts_100g`
    """
    if x <= 40:
        return 0
    elif x <= 60:
        return 2
    elif x <= 80:
        return 4
    else:
        return 10
    return x

<img src='./images/logo_OC.png' width=20px align="left"/> &emsp; Formule de calcul du  **score** : `pts_A`

In [77]:
def pts_A_func(pts_kj, pts_glus, pts_ags, pts_na):
    """
    Cette fonction, calcul le score `pts_A` qui est l'agrégation des scores `pts_kj`, `pts_glus`, `pts_ags` et `pts_na`
    Cette fonction sera utlisée pour tous les produits, sauf les produits de la famille Added Fat.
    """
    pts_A = pts_kj + pts_glus + pts_ags + pts_na
    return pts_A

<img src='./images/logo_OC.png' width=20px align="left"/> &emsp; Formule de calcul du  **score** : `pts_A_fat`

In [78]:
def pts_A_func_fat(pts_kj, pts_glus, pts_ags_fat, pts_na):
    """
    Cette fonction, calcul le score `pts_A_fat` qui est l'agrégation des scores `pts_kj`, `pts_glus`, `pts_ags_fat` et `pts_na`
    Cette fonction sera utlisée UNIQUEMENT pour les produits de la famille Added Fat.
    """
    pts_A_fat = pts_kj + pts_glus + pts_ags_fat + pts_na
    return pts_A_fat

<img src='./images/logo_OC.png' width=20px align="left"/> &emsp; Formule de calcul du  **score** : `pts_A_Beverages`

In [79]:
def pts_A_func_Beverages(pts_kj_Beverages, pts_glus_Beverages, pts_ags, pts_na):
    """
    Cette fonction, calcul le score `pts_A_fat` qui est l'agrégation des scores `pts_kj`, `pts_glus`, `pts_ags_fat` et `pts_na`
    Cette fonction sera utlisée UNIQUEMENT pour les produits de la famille Added Fat.
    """
    pts_A_Beverage = pts_kj_Beverages + pts_glus_Beverages + pts_ags + pts_na

    return pts_A_Beverage

<img src='./images/logo_OC.png' width=20px align="left"/> &emsp; Formule de calcul du  **score** : `pts_AGS_lip_tot`

In [80]:
def pts_AGS_Lip_tot(saturated_fat_100g, fat_100g):
    """
    Cette fonction, calcul le score `pts_AGS_lip_tot` qui est l'agrégation des scores des variables `saturated-fat-100g` et `fat-100g`
    Cette fonction sera utlisée pour tous les produits, sauf les produits de la famille Added Fat.
    """
    if fat_100g == 0:
        AGS_Lip_tot = 0
    else:
        AGS_Lip_tot = (saturated_fat_100g / fat_100g) * 100

    return AGS_Lip_tot

<img src='./images/logo_OC.png' width=20px align="left"/> &emsp; Formule de calcul du **Nutri-Score**.

In [81]:
def Nutri_Score_func(pts_A, pts_prot, pts_fib, pts_FLN):
    """
    Cette fonction, calcul le nutri-score pour tous les produits sauf ceux qui appartient aux catégories : Added Fat ou Cheese.
    """
    if pts_A < 11:
        return pts_A - (pts_prot + pts_fib + pts_FLN)

    elif (pts_A >= 11) & (pts_FLN >= 5):
        return pts_A - (pts_prot + pts_fib + pts_FLN)

    else:
        return pts_A - (pts_fib + pts_FLN)

    return nutri_score_calculated

<img src='./images/logo_OC.png' width=20px align="left"/> &emsp; Formule de calcul du **Nutri-Score** pour les produits de la famille **Added Fat**.

In [82]:
def Nutri_Score_func_fat(pts_A_fat, pts_prot, pts_fib, pts_FLN):
    """
    Cette fonction, calcul le nutri-score UNIQUEMENT pour les produits de la famille: Added Fat.
    """
    if pts_A_fat < 11:
        return pts_A_fat - pts_prot - pts_fib - pts_FLN

    elif (pts_A_fat >= 11) & (pts_FLN == 5):
        return pts_A_fat - pts_prot - pts_fib - pts_FLN

    else:
        return pts_A_fat - pts_fib - pts_FLN

    return nutri_score_calculated

<img src='./images/logo_OC.png' width=20px align="left"/> &emsp;  Formule de calcul du **Nutri-Score** pour les produits de la famille `Beverages` **UNIQUEMENT**.

In [83]:
def Nutri_Score_func_Beverages(pts_A_Beverage, pts_prot, pts_fib, pts_FLN_Beverages):
    """
    Cette fonction, calcul le nutri-score UNIQUEMENT pour les produits de la famille: Cheese.
    """
    if pts_A_Beverage < 11:
        return pts_A_Beverage - pts_prot - pts_fib - pts_FLN_Beverages

    elif (pts_A_Beverage >= 11) & (pts_FLN_Beverages > 5):
        return pts_A_Beverage - pts_prot - pts_fib - pts_FLN_Beverages

    else:
        return pts_A_Beverage - pts_fib - pts_FLN_Beverages

    return nutri_score_calculated

<img src='./images/logo_OC.png' width=20px align="left"/> &emsp;  Formule de calcul du **Nutri-Score** pour les produits de la famille `Cheese` **UNIQUEMENT**.

In [84]:
def Nutri_Score_func_Cheese_Case(pts_A, pts_prot, pts_fib, pts_FLN):
    """
    Cette fonction, calcul le nutri-score UNIQUEMENT pour les produits de la famille: Cheese.
    """
    nutri_score_calculated = pts_A - (pts_prot + pts_fib + pts_FLN)

    return nutri_score_calculated

<img src='./images/logo_OC.png' width=20px align="left"/> &emsp; Formule de calcul du **Nutri-Grade** en fonction du résultat du **Nutri-Score**

In [85]:
def Nutri_Grade_func(x):
    """
    Cette fonction, calcul le nutri-grade pour tous les produits sauf ceux qui appartient à la catégorie : Beverages.
    """
    if x < 0:
        return "a"
    elif x < 3:
        return "b"
    elif x < 11:
        return "c"
    elif x < 19:
        return "d"
    else:
        return "e"
    return x

<img src='./images/logo_OC.png' width=20px align="left"/> &emsp; Formule de calcul du **Nutri-Grade** en fonction du résultat du **Nutri-Score** pour des produits de la catégorie `Beverage` **UNIQUEMENT**.

In [86]:
def Nutri_Grade_func_Beverage(x, y):
    """
    Cette fonction, calcul le nutri-grade UNIQUEMENT pour les produits de la famille: Beverages.
    """
    if x == "Waters and flavored waters":
        grade = "a"
    elif y < 2:
        grade = "b"
    elif y < 6:
        grade = "c"
    elif y < 10:
        grade = "d"
    else:
        grade = "e"
    return grade

<img src='./images/logo_OC.png' width=20px align="left"/> &emsp; LA fonction **MAIN** calculer le **Nutri-Score** et **Nutri-Grade** pour chaque catégorie de produit. ` version 1.0`

In [87]:
# def main_Nutri_and_Grade_Score(dataframe) :

# #"""
# version 1.0,
# Dérnière vérsion : NON.
# #Cas n°1 sur 4 : Fat
# #"""

#     if dataframe.loc[:,["pnns_groups_1"]] in np.array(["Fat and sauces"]) :
#                 dataframe["pts_kj"] = dataframe["energy_100g"].apply(pts_kj_func)
#                 dataframe["pts_glus"] = dataframe["sugars_100g"].apply(pts_glus_func)
#                 dataframe["AGS_Lip_tot"] = ((dataframe["saturated-fat_100g"] /df_test4["fat_100g"]) * 100).fillna(0)
#                 dataframe["pts_ags"] = dataframe["AGS_Lip_tot"].apply(pts_ags_func)
#                 dataframe["pts_na"] = dataframe["sodium_100mg"].apply(pts_na_func)
#                 dataframe["pts_prot"] = dataframe["proteins_100g"].apply(pts_prot_func)
#                 dataframe["pts_fib"] = dataframe["fiber_100g"].apply(pts_fib_func)
#                 dataframe["pts_FLN"] = dataframe["fruits-vegetables-nuts_100g"].apply(pts_FLN_func)
#                 dataframe["pts_A"] = dataframe["pts_kj"] + dataframe["pts_glus"] + dataframe["pts_ags"] + dataframe["pts_na"]
#                 dataframe["nutriscore_score_calculated"] = dataframe[["pts_A","pts_prot","pts_fib","pts_FLN"]].apply(lambda x : Nutri_Score_func(*x), axis=1).astype(float)
#                 dataframe["nutriscore_grade_calculated"] = dataframe["nutriscore_score_calculated"].apply(Nutri_Grade_func)

# #"""
# #Cas n°2 sur 4 : Beverages
# #"""

#     elif dataframe.loc[:,["pnns_groups_1"]]  in np.array(["Beverages"]) :
#                 dataframe["pts_kj"] = dataframe["energy_100g"].apply(pts_kj_func)
#                 dataframe["pts_glus"] = dataframe["sugars_100g"].apply(pts_glus_func)
#                 dataframe["pts_ags"] = dataframe["saturated-fat_100g"].apply(pts_ags_func_Beverage_Case)
#                 dataframe["pts_na"] = dataframe["sodium_100mg"].apply(pts_na_func)
#                 dataframe["pts_prot"] = dataframe["proteins_100g"].apply(pts_prot_func)
#                 dataframe["pts_fib"] = dataframe["fiber_100g"].apply(pts_fib_func)
#                 dataframe["pts_FLN"] = dataframe["fruits-vegetables-nuts_100g"].apply(pts_FLN_func)
#                 dataframe["pts_A"] = dataframe["pts_kj"] + dataframe["pts_glus"] + dataframe["pts_ags"] + dataframe["pts_na"]
#                 dataframe["nutriscore_score_calculated"] = dataframe[["pts_A","pts_prot","pts_fib","pts_FLN"]].apply(lambda x : Nutri_Score_func(*x), axis=1).astype(float)
#                 dataframe["nutriscore_grade_calculated"] = dataframe["nutriscore_score_calculated"].apply(Nutri_Grade_func_Beverage_Case)

# #"""
# #Cas n°3 sur 4 : Cheese
# #"""
#     elif  dataframe.loc[:,["pnns_groups_1"]] in np.array(["Milk and dairy products"]) :
#                 dataframe['pts_kj'] = dataframe['energy_100g'].apply(pts_kj_func)
#                 dataframe['pts_glus'] = dataframe['sugars_100g'].apply(pts_glus_func)
#                 dataframe["pts_ags"] = dataframe["saturated-fat_100g"].apply(pts_ags_func_Beverage_Case)
#                 dataframe["pts_na"] = dataframe["sodium_100mg"].apply(pts_na_func)
#                 dataframe["pts_prot"] = dataframe["proteins_100g"].apply(pts_prot_func)
#                 dataframe["pts_fib"] = dataframe["fiber_100g"].apply(pts_fib_func)
#                 dataframe["pts_FLN"] = dataframe["fruits-vegetables-nuts_100g"].apply(pts_FLN_func)
#                 dataframe["pts_A"] = (dataframe["pts_kj"] + dataframe["pts_glus"] + dataframe["pts_ags"] + dataframe["pts_na"])
#                 dataframe["nutriscore_score_calculated"] = [Nutri_Score_func_Cheese_Case(A,prot,fib,FLN) for A, prot, fib, FLN in zip(dataframe["pts_A"], dataframe["pts_prot"],dataframe["pts_fib"],dataframe["pts_FLN"])]
# #                 dataframe["nutriscore_score_calculated"] = dataframe[["pts_A","pts_prot","pts_fib","pts_FLN"]].apply(lambda x: Nutri_Score_func_Cheese_Case(*x), axis=1).astype(float)
#                 dataframe["nutriscore_grade_calculated"] = dataframe["nutriscore_score_calculated"].apply(Nutri_Grade_func)
# #"""
# #Cas n°4 sur 4 : General cas
# #"""
#     else :
#                 dataframe["pts_kj"] = dataframe["energy_100g"].apply(pts_kj_func)
#                 dataframe["pts_glus"] = dataframe["sugars_100g"].apply(pts_glus_func)
#                 dataframe["pts_ags"] = dataframe["saturated-fat_100g"].apply(pts_ags_func_Beverage_Case)
#                 dataframe["pts_na"] = dataframe["sodium_100mg"].apply(pts_na_func)
#                 dataframe["pts_prot"] = dataframe["proteins_100g"].apply(pts_prot_func)
#                 dataframe["pts_fib"] = dataframe["fiber_100g"].apply(pts_fib_func)
#                 dataframe["pts_FLN"] = dataframe["fruits-vegetables-nuts_100g"].apply(pts_FLN_func)
#                 dataframe["pts_A"] = dataframe["pts_kj"] + dataframe["pts_glus"] + dataframe["pts_ags"] + dataframe["pts_na"]
#                 dataframe["nutriscore_score_calculated"] = dataframe[["pts_A","pts_prot","pts_fib","pts_FLN"]].apply(lambda x : Nutri_Score_func(*x), axis=1).astype(float)
#                 dataframe["nutriscore_grade_calculated"] = dataframe["nutriscore_score_calculated"].apply(Nutri_Grade_func)

# #     dataframe.drop(["pts_kj","pts_glus","pts_ags","pts_na","pts_prot","pts_fib","pts_FLN","pts_A","AGS_Lip_tot"], axis=1, inplace=True)

#       return dataframe

La fonction **MAIN** `version 1.0` a été **<font color=red>abandonnées</font>**, car elle ne respecte pas les recommandations du guide d'utilisation de Pandas : (https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy)

<img src='./images/logo_OC.png' width=20px align="left"/> &emsp; LA fonction **MAIN** pour calculer le **Nutri-Score**  ` version 2.0`

In [88]:
def main_Nutri_Score(
    energy_100g,
    sugars_100g,
    saturated_fat_100g,
    fat_100g,
    sodium_100mg,
    proteins_100g,
    fiber_100g,
    fruits_vegetables_nuts_100g,
    pnns_groups_1,
):

    pts_kj = pts_kj_func(energy_100g)
    pts_kj_Beverages = pts_kj_func_Beverages(energy_100g)
    pts_glus = pts_glus_func(sugars_100g)
    pts_glus_Beverages = pts_glus_func_Beverages(sugars_100g)
    pts_ags = pts_ags_func(saturated_fat_100g)
    AGS_Lip_tot = pts_AGS_Lip_tot(saturated_fat_100g, fat_100g)
    pts_ags_fat = pts_ags_func_fat(AGS_Lip_tot)
    pts_na = pts_na_func(sodium_100mg)
    pts_prot = pts_prot_func(proteins_100g)
    pts_fib = pts_fib_func(fiber_100g)
    pts_FLN = pts_FLN_func(fruits_vegetables_nuts_100g)
    pts_FLN_Beverages = pts_FLN_func_Beverages(fruits_vegetables_nuts_100g)
    pts_A = pts_A_func(pts_kj, pts_glus, pts_ags, pts_na)
    pts_A_Beverage = pts_A_func_Beverages(
        pts_kj_Beverages, pts_glus_Beverages, pts_ags, pts_na
    )
    pts_A_fat = pts_A_func_fat(pts_kj, pts_glus, pts_ags_fat, pts_na)

    if pnns_groups_1 == "Fat and sauces":
        nutri_score_calculated = Nutri_Score_func_fat(
            pts_A_fat, pts_prot, pts_fib, pts_FLN
        )
    elif pnns_groups_1 == "Milk and dairy products":
        nutri_score_calculated = Nutri_Score_func_Cheese_Case(
            pts_A, pts_prot, pts_fib, pts_FLN
        )
    elif pnns_groups_1 == "Beverages":
        nutri_score_calculated = Nutri_Score_func_Beverages(
            pts_A_Beverage, pts_prot, pts_fib, pts_FLN_Beverages
        )
    else:
        nutri_score_calculated = Nutri_Score_func(pts_A, pts_prot, pts_fib, pts_FLN)

    return nutri_score_calculated

<img src='./images/logo_OC.png' width=20px align="left"/> &emsp; LA fonction **MAIN** pour calculer le **Nutri-Grade**  ` version 2.0`

In [89]:
def main_Nutri_Grade(pnns_groups_1, pnns_groups_2, nutriscore_score_calculated):

    if pnns_groups_1 == "Fat and sauces":
        nutri_grade_calculated = Nutri_Grade_func(nutriscore_score_calculated)

    elif pnns_groups_1 == "Milk and dairy products":
        nutri_grade_calculated = Nutri_Grade_func(nutriscore_score_calculated)

    elif pnns_groups_1 == "Beverages":
        nutri_grade_calculated = Nutri_Grade_func_Beverage(
            pnns_groups_2, nutriscore_score_calculated
        )
    else:
        nutri_grade_calculated = Nutri_Grade_func(nutriscore_score_calculated)

    return nutri_grade_calculated

<img src='./images/logo_OC.png' width=20px align="left"/> &emsp; Test de **MAIN** pour calculer le **Nutri-Score**.

In [90]:
df_app_test = df_app.dropna().copy()

In [91]:
a = round(
    len(df_app.dropna()[df_app_test["pnns_groups_1"] == "unknown"])
    / len(df_app_test)
    * 100,
    2,
)
print(f"- {a}% des produits du dataset n'ont pas de catégorie définie : unknown")

- 2.58% des produits du dataset n'ont pas de catégorie définie : unknown


In [92]:
df_app_test.dtypes.sort_values()

fat_100g                       float64
nutriscore_score               float64
fruits-vegetables-nuts_100g    float64
fiber_100g                     float64
salt_100g                      float64
proteins_100g                  float64
sugars_100g                    float64
saturated-fat_100g             float64
sodium_100mg                   float64
additives_n                    float64
energy_100g                    float64
nutriscore_grade                object
manufacturing_places            object
countries                       object
pnns_groups_2                   object
pnns_groups_1                   object
product_name                    object
url                             object
code                            object
dtype: object

In [93]:
df_app_test["nutri_score_calculated"] = df_app_test[
    [
        "energy_100g",
        "sugars_100g",
        "saturated-fat_100g",
        "fat_100g",
        "sodium_100mg",
        "proteins_100g",
        "fiber_100g",
        "fruits-vegetables-nuts_100g",
        "pnns_groups_1",
    ]
].apply(lambda x: main_Nutri_Score(*x), axis=1)

<img src='./images/logo_OC.png' width=20px align="left"/> &emsp; `Accuracy` de la fonction `Main Nutri-Score`

In [94]:
print(
    "La fiabilité de la fonction MAIN nutri-score est de :",
    round(
        (
            sum(
                df_app_test["nutriscore_score"] == df_app_test["nutri_score_calculated"]
            )
            / len(df_app_test["nutriscore_score"])
        )
        * 100,
        2,
    ),
    "%",
)
print("--" * 50)
print(
    "Le gap avec LA fiabilité MAX est de ",
    round(
        98.13
        - (
            sum(
                df_app_test["nutriscore_score"] == df_app_test["nutri_score_calculated"]
            )
            / len(df_app_test["nutriscore_score"])
        )
        * 100,
        2,
    ),
    "%",
)

La fiabilité de la fonction MAIN nutri-score est de : 94.77 %
----------------------------------------------------------------------------------------------------
Le gap avec LA fiabilité MAX est de  3.36 %


In [95]:
df_app_test["nutri_grade_calculated"] = df_app_test[
    ["pnns_groups_1", "pnns_groups_2", "nutri_score_calculated"]
].apply(lambda x: main_Nutri_Grade(*x), axis=1)

<img src='./images/logo_OC.png' width=20px align="left"/> &emsp; `Accuracy` de la fonction `Main Nutri-Grade`

In [96]:
print(
    "La fiabilité de la fonction MAIN nutri-grade est de :",
    round(
        (
            sum(
                df_app_test["nutriscore_grade"] == df_app_test["nutri_grade_calculated"]
            )
            / len(df_app_test["nutriscore_grade"])
        )
        * 100,
        2,
    ),
    "%",
)
print("--" * 50)
print(
    "Le gap avec LA fiabilité MAX est de ",
    round(
        98.13
        - (
            sum(
                df_app_test["nutriscore_grade"] == df_app_test["nutri_grade_calculated"]
            )
            / len(df_app_test["nutriscore_grade"])
        )
        * 100,
        2,
    ),
    "%",
)

La fiabilité de la fonction MAIN nutri-grade est de : 97.22 %
----------------------------------------------------------------------------------------------------
Le gap avec LA fiabilité MAX est de  0.91 %


In [97]:
# DF_Nutri_Score = df_app_test.copy()

In [98]:
# DF_Nutri_Score["pts_kj"] = DF_Nutri_Score["energy_100g"].apply(lambda x : pts_kj_func(x))
# DF_Nutri_Score["pts_kj_Beverages"] = DF_Nutri_Score["energy_100g"].apply(lambda x : pts_kj_func_Beverages(x))
# DF_Nutri_Score["pts_glus"] = DF_Nutri_Score["sugars_100g"].apply(lambda x : pts_glus_func(x))
# DF_Nutri_Score["pts_glus_Beverages"] = DF_Nutri_Score["sugars_100g"].apply(lambda x : pts_glus_func_Beverages(x))
# DF_Nutri_Score["pts_ags"] = DF_Nutri_Score["saturated-fat_100g"].apply(lambda x : pts_ags_func(x))
# DF_Nutri_Score["AGS_Lip_tot"] = DF_Nutri_Score[["saturated-fat_100g","fat_100g"]].apply(lambda x : pts_AGS_Lip_tot(*x), axis=1)
# DF_Nutri_Score["pts_ags_fat"] = DF_Nutri_Score["AGS_Lip_tot"].apply(lambda x : pts_ags_func_fat(x))
# DF_Nutri_Score["pts_na"] = DF_Nutri_Score["sodium_100mg"].apply(lambda x : pts_na_func(x))
# DF_Nutri_Score["pts_prot"] = DF_Nutri_Score["proteins_100g"].apply(lambda x : pts_prot_func(x))
# DF_Nutri_Score["pts_fib"] = DF_Nutri_Score["fiber_100g"].apply(lambda x : pts_fib_func(x))
# DF_Nutri_Score["pts_FLN"] = DF_Nutri_Score["fruits-vegetables-nuts_100g"].apply(lambda x : pts_FLN_func(x))
# DF_Nutri_Score["pts_FLN_Beverages"] = DF_Nutri_Score["fruits-vegetables-nuts_100g"].apply(lambda x : pts_FLN_func_Beverages(x))
# DF_Nutri_Score["pts_A"] = DF_Nutri_Score[["pts_kj","pts_glus","pts_ags","pts_na"]].apply(lambda x : pts_A_func(*x), axis=1)
# DF_Nutri_Score["pts_A_fat"] = DF_Nutri_Score[["pts_kj","pts_glus","pts_ags_fat","pts_na"]].apply(lambda x : pts_A_func_fat(*x), axis=1)
# DF_Nutri_Score["pts_A_Beverage"] = DF_Nutri_Score[["pts_kj_Beverages","pts_glus_Beverages","pts_ags","pts_na"]].apply(lambda x : pts_A_func_Beverages(*x), axis=1)

In [99]:
# DF_Nutri_Score["nutriscore_score_calculated"] = DF_Nutri_Score[["energy_100g",
#                                                           "sugars_100g",
#                                                           "saturated-fat_100g",
#                                                           "fat_100g",
#                                                           "sodium_100mg",
#                                                           "proteins_100g",
#                                                           "fiber_100g",
#                                                           "fruits-vegetables-nuts_100g",
#                                                           "pnns_groups_1"]].apply(lambda x : main_Nutri_Score(*x), axis=1)

In [100]:
# DF_Nutri_Score["nutriscore_grade_calculated"] = DF_Nutri_Score[["pnns_groups_1",
#                                                           "pnns_groups_2",
#                                                           "nutriscore_score_calculated"]].apply(lambda x : main_Nutri_Grade(*x), axis=1)

In [101]:
# (sum(DF_Nutri_Score["nutriscore_score"] == DF_Nutri_Score["nutriscore_score_calculated"]) / len(DF_Nutri_Score["nutriscore_score"])) * 100

In [102]:
# (sum(DF_Nutri_Score["nutriscore_grade"] == DF_Nutri_Score["nutriscore_grade_calculated"]) / len(DF_Nutri_Score["nutriscore_grade"])) * 100

In [103]:
# DF_Nutri_Score[(DF_Nutri_Score["nutriscore_score"] != DF_Nutri_Score["nutriscore_score_calculated"]) & (~DF_Nutri_Score["pnns_groups_1"].isin(np.array(["Beverages","Milk and dairy products","Fat and sauces"])))]

In [104]:
# DF_Nutri_Score[(DF_Nutri_Score["pnns_groups_1"] =="Fat and sauces") & (DF_Nutri_Score["nutriscore_grade"] == "a")]

In [105]:
# DF_Nutri_Score.loc[(DF_Nutri_Score["nutriscore_score"] != DF_Nutri_Score["nutriscore_score_calculated"]) & (DF_Nutri_Score["pnns_groups_1"] =="Milk and dairy products"),["pnns_groups_1","pnns_groups_2"]].drop_duplicates()

### <img src='./images/logo_OC.png' width=20px align="left"/> &emsp; Imputation des valeurs manquantes sur les variables quantitatives (sauf le nutri-score  et le nutri-grade).

<img src='./images/logo_OC.png' width=20px align="left"/> &emsp; Préparer l'imputation des colonnes numériques **UNIQUEMENT** sauf la colonne  `nutri-score`

In [106]:
index_list = [
    "code",
    "url",
    "product_name",
    "pnns_groups_1",
    "pnns_groups_2",
    "countries",
    "manufacturing_places",
    "nutriscore_score",
    "nutriscore_grade",
]

In [107]:
df_app_Imputed = df_app.copy()

In [108]:
df_app_Imputed = df_app_Imputed.set_index(index_list).drop("sodium_100mg", axis=1)

In [109]:
df_app_Imputed.head(2)

,,,,,,,,,additives_n,energy_100g,fat_100g,saturated-fat_100g,sugars_100g,proteins_100g,salt_100g,fiber_100g,fruits-vegetables-nuts_100g
code,url,product_name,pnns_groups_1,pnns_groups_2,countries,manufacturing_places,nutriscore_score,nutriscore_grade,,,,,,,,,
0000000005,http://world-en.openfoodfacts.org/product/0000000005/nectar-d-abricot-ferme-de-l-aiguemarse,Nectar d'abricot,Beverages,Sweetened beverages,en:France,Not Made in France,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0000000020114,http://world-en.openfoodfacts.org/product/0000000020114/naturablue-original-natura4ever,Naturablue original,unknown,unknown,France,Made in France,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN


<img src='./images/logo_OC.png' width=20px align="left"/> &emsp; Compter le nombre de valeurs manquantes par colonne **AVANT**  imputation avec `KNNImputer`

In [110]:
round(df_app_Imputed.isna().mean().sort_values(ascending=False) * 100, 2)

fruits-vegetables-nuts_100g    98.16
fiber_100g                     54.63
saturated-fat_100g             19.00
sugars_100g                    18.11
salt_100g                      17.72
energy_100g                    16.75
proteins_100g                  16.55
fat_100g                       16.20
additives_n                     9.76
dtype: float64

In [111]:
impute_knn = KNNImputer(n_neighbors=5)

In [112]:
%%time
df_app_Imputed = pd.DataFrame(
    impute_knn.fit_transform(df_app_Imputed),
    index=df_app_Imputed.index,
    columns=df_app_Imputed.columns,
)

Wall time: 21min 57s


<img src='./images/logo_OC.png' width=20px align="left"/> &emsp; Compter le nombre de valeurs manquantes par colonne **APRÈS** imputation avec `KNNImputer`


In [113]:
round(df_app_Imputed.isna().mean().sort_values(ascending=False) * 100, 2)

additives_n                    0.0
energy_100g                    0.0
fat_100g                       0.0
saturated-fat_100g             0.0
sugars_100g                    0.0
proteins_100g                  0.0
salt_100g                      0.0
fiber_100g                     0.0
fruits-vegetables-nuts_100g    0.0
dtype: float64

In [114]:
df_app_Imputed.reset_index(inplace=True)

In [115]:
re_ordered_cols = [
    "code",
    "url",
    "product_name",
    "additives_n",
    "pnns_groups_1",
    "pnns_groups_2",
    "countries",
    "manufacturing_places",
    "energy_100g",
    "fat_100g",
    "saturated-fat_100g",
    "sugars_100g",
    "proteins_100g",
    "salt_100g",
    "fiber_100g",
    "fruits-vegetables-nuts_100g",
    "nutriscore_score",
    "nutriscore_grade",
]

In [116]:
df_app_Imputed = df_app_Imputed[re_ordered_cols]

In [117]:
df_app_Imputed.head()

,code,url,product_name,additives_n,pnns_groups_1,pnns_groups_2,countries,manufacturing_places,energy_100g,fat_100g,saturated-fat_100g,sugars_100g,proteins_100g,salt_100g,fiber_100g,fruits-vegetables-nuts_100g,nutriscore_score,nutriscore_grade
0,0000000005,http://world-en.openfoodfacts.org/product/0000...,Nectar d'abricot,0.0,Beverages,Sweetened beverages,en:France,Not Made in France,595.4,13.474,2.86,2.56,6.49,0.742000,2.298,58.00,NaN,NaN
1,0000000020114,http://world-en.openfoodfacts.org/product/0000...,Naturablue original,1.0,unknown,unknown,France,Made in France,0.0,0.000,0.00,0.00,0.00,9.006604,0.000,10.00,NaN,NaN
2,0000000274722,http://world-en.openfoodfacts.org/product/0000...,Blanquette de Volaille et son Riz,2.0,Composite foods,One-dish meals,France,Made in France,450.0,2.200,0.90,0.50,6.80,0.700000,0.500,44.52,0.0,b
3,0000000290616,http://world-en.openfoodfacts.org/product/0000...,Salade Cesar,3.0,Fruits and vegetables,Vegetables,Canada,Not Made in France,1210.0,12.000,7.00,0.00,22.00,2.160000,2.000,44.60,6.0,c
4,0000000394710,http://world-en.openfoodfacts.org/product/0000...,Danoises à la cannelle roulées,8.0,Sugary snacks,Biscuits and cakes,Canada,Not Made in France,1520.0,14.400,4.92,28.10,4.79,0.922000,2.050,26.68,NaN,NaN


In [118]:
df_app_Imputed.isna().sum().sort_values()

code                               0
fruits-vegetables-nuts_100g        0
fiber_100g                         0
salt_100g                          0
proteins_100g                      0
sugars_100g                        0
saturated-fat_100g                 0
fat_100g                           0
energy_100g                        0
manufacturing_places               0
countries                          0
pnns_groups_2                      0
additives_n                        0
url                                0
pnns_groups_1                     19
product_name                    1107
nutriscore_score               29875
nutriscore_grade               29875
dtype: int64

### <img src='./images/logo_OC.png' width=20px align="left"/> &emsp; Trouver la meilleure value du paramètre : `n_neighbors` pour la fonction `KNNImputer`.

In [119]:
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression

In [120]:
df_app_numerical_data_only = df_app.select_dtypes(include=[int, float]).dropna().copy()

In [121]:
len(df_app_numerical_data_only)

1549

In [122]:
df_app_numerical_data_only.head(2)

,additives_n,energy_100g,fat_100g,saturated-fat_100g,sugars_100g,proteins_100g,salt_100g,fiber_100g,fruits-vegetables-nuts_100g,nutriscore_score,sodium_100mg
920,11.0,1670.0,15.0,2.0,34.0,4.9,0.60,1.5,37.4,13.0,240.0
2242,2.0,2030.0,26.8,7.6,29.4,6.0,0.37,1.9,12.0,19.0,148.0


In [123]:
X = df_app_numerical_data_only.drop("nutriscore_score", axis=1)
y = df_app_numerical_data_only["nutriscore_score"]

In [124]:
model = make_pipeline(KNNImputer(), LinearRegression())

In [125]:
params = {"knnimputer__n_neighbors": list(range(2, 13))}

In [126]:
grid = GridSearchCV(model, param_grid=params, cv=5)

In [127]:
grid.fit(X, y)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('knnimputer', KNNImputer()),
                                       ('linearregression',
                                        LinearRegression())]),
             param_grid={'knnimputer__n_neighbors': [2, 3, 4, 5, 6, 7, 8, 9, 10,
                                                     11, 12]})

In [128]:
grid.best_params_

{'knnimputer__n_neighbors': 2}

* La meilleure valeur du paramètre `n_neighbors` pour `KNNImputer() est de **2**. C'est cette valeur que nous allons utiliser pour le reste du projet.

### <img src='./images/logo_OC.png' width=20px align="left"/> &emsp; Création d'une fonction pour **AUTOMATISER** l'imputation des valeurs manquantes avec `KNNImputer`

In [129]:
def numerical_Values_Imputation(dataframe, index_list):
    """ """

    # Liste des colonnes non concernées par l'imputation avec KNNImputer
    #     index_list = ["code",
    #                   "url",
    #                   "product_name",
    #                   "pnns_groups_1",
    #                   "pnns_groups_2",
    #                   "brands",
    #                   "countries",
    #                   "manufacturing_places",
    #                   "stores",
    #                   "nutriscore_score",
    #                   "nutriscore_grade",
    #                    ]

    # Deplacer les colonnes non concernées par l'imputation avec KNNImputer en indices.
    # df_app_Imputed = dataframe.copy()
    dataframe = dataframe.set_index(index_list)

    # Compter le nombre de valeurs manquantes par colonne **AVANT**  imputation avec `KNNImputer.
    print(
        "Le nombre de valeurs manquantes trouvées AVANT imputation : ",
        dataframe.isna().sum().sum(),
    )

    # Initialisation de `KNNImputer.
    impute_knn = KNNImputer(n_neighbors=2)

    # Imputation des valeurs manquantes.
    df_imputed = pd.DataFrame(
        impute_knn.fit_transform(dataframe),
        index=dataframe.index,
        columns=dataframe.columns,
    )

    # Compter le nombre de valeurs manquantes par colonne **APRES**  imputation avec `KNNImputer.
    print(
        "Le nombre de valeurs manquantes trouvées APRES imputation : ",
        df_imputed.isna().sum().sum(),
    )

    # Deplacer les colonnes non concernées par l'imputation vers leur place initiale.
    df_imputed.reset_index(inplace=True)

    print("Imputation des valeurs manquantes réalisée avec succès")

    return df_imputed

*********
#  <font color=#7451eb> Chapitre 4 : Automatiser le nettoyage du dataset</font>
*********

### <img src='./images/logo_OC.png' width=20px align="left"/> &emsp; Création d'une fonction pour **AUTOMATISER tous les traitements** pour nettoyer le dataset.

In [130]:
def dataset_auto_clean(input_file_path, out_file_path):
    """
    Cette fonction retourne un dataframe après avoir réalisé les opérations suivantes :
    - Etape 01 : Lire le dataframe, supprimer les colonnes qui ne sont pas en lien avec l'idée de l'application.
    - Etape 02 : Définir les bons Dtypes pour les différentes colonnes.
    - Etape 03 : Chercher et corriger les Fake NaN.
    - Etape 04 : Supprimer les valeurs en double sur la colonne "code".
    - Etape 05 : Nettoyage des colonnes "countries" et "manufacturing_places" + supprimer les valeurs manquantes.
    - Etape 06 : Nettoyage des colonnes "XXXX","XXXXXX", "XXXXXXXXX" et "XXXXXXXX" + supprimer les valeurs manquantes.
    - Etape 07 : Imputation des valeurs manquantes sur toutes les colonnes numériques SAUF la colonne "nutri-score".
    - Etape 08 : Créer une nouvelle colonne "sodium_10mg", calculer les valeurs de cette colonne à partir de la colonne "salt_10g".
    - Etape 09 : Calcul des valeurs manquantes sur la colonne "nutri-score" et "nutri-grade".
    - Etape 10 : Enregistrer le dataframe nettoyé.
    """
    #  Etape 01 : Lire le dataframe.
    print("-" * 100)
    print("Etape 01 : Charger le dataset.")

    app_idea_Columns = [
        "code",
        "url",
        "product_name",
        "pnns_groups_1",
        "pnns_groups_2",
        "additives_n",
        "countries",
        "manufacturing_places",
        "energy_100g",
        "fat_100g",
        "saturated-fat_100g",
        "sugars_100g",
        "proteins_100g",
        "salt_100g",
        "fiber_100g",
        "fruits-vegetables-nuts_100g",
        "nutriscore_score",
        "nutriscore_grade",
    ]

    try:

        df_app = pd.read_csv(
            input_file_path, sep="\t", usecols=app_idea_Columns, low_memory=False
        )

        print("le dataset a été correctement chargé.")

    except:

        print(
            "Une erreur s'est produite lors de la lecture du dataset.\nMerci de vérifier les points suivantes:\n1. Le fichier est bien enregistré sous le dossier 03_Project_03/data/.\n2. Le fichier est bien au format .csv"
        )

    print("-" * 100)

    #  Etape 02 : Définir les bons Dtypes pour les différentes colonnes.
    print("Etape 02 : Définir les bons Dtypes pour les différentes colonnes")
    #  Appeler la fonction mixte_type_conversion, pour définir le bon Dtype pour chaque colonne.

    try:

        mixte_type_conversion(df_app)

    except:

        print("Aucune conversion n'est nécessaire des Dtypes.")

    print("-" * 100)

    # Etape 03 : Chercher et corriger les Fake NaN.
    print("Etape 03 : Chercher et corriger les Fake NaN")

    if np.sum(df_app == "nan").sum() > 0:
        print(
            "Le nombre de fake NaN détecté AVANT correction: ",
            np.sum(df_app == "nan").sum(),
        )
        df_app = df_app.applymap(correct_fake_NaN)
        print(
            "Le nombre de fake NaN détecté APRES correction: ",
            np.sum(df_app == "nan").sum(),
        )
    else:
        print("Aucune valeur Fake NaN n'a été détectée")

    print("-" * 100)

    # Etape 04 : Supprimer les valeurs en double sur la colonne "code".
    print("Etape 04 : Suppression des valeurs en double sur la colonne code.")

    print(
        "Le nombre de valeurs en double détectées sur le colonne code AVANT correction : ",
        df_app.duplicated(subset="code").sum(),
    )
    df_app = df_app.drop_duplicates(subset=["code"])
    print(
        "Le nombre de valeurs en double détectées sur le colonne code APRES correction : ",
        df_app.duplicated(subset="code").sum(),
    )

    print("-" * 100)

    # Etape 05 : Nettoyage des colonnes "countries" et "manufacturing_places" + supprimer les valeurs manquantes.
    # Etape 05 - 01 : Nettoyage de la colonne "countries".
    print("Etape 05 : Nettoyage des colonnes countries et manufacturing_places.")

    df_app.dropna(subset=["countries"], inplace=True)

    Produits_Vendus_En_France = [
        "en:france",
        "France",
        "france",
        "FRANCE",
        "en:France",
        "en:fr",
        "en:FR",
        "Frankreich",
        "Francia",
        "La Réunion",
        "Réunion",
        "French Polynesia",
        "french-polynesia",
        "New Caledonia",
        "Polynésie française",
        "Guadeloupe",
        "Frankrijk",
        "Martinique",
        "martinique",
        "França",
        "new-caledonia",
        "Nouvelle-Calédonie",
        "Monaco",
        "Frantsa",
        "Fransa",
        "Francia",
        "Mayotte",
        "en:Mayotte",
    ]

    df_app.loc[
        df_app["countries"].isin(Produits_Vendus_En_France), ["countries"]
    ] = "France"
    df_app = df_app.drop(df_app[df_app["countries"] != "France"].index)

    print("Nettoyage de la colonne `countries` OK")

    # Etape 05 - 02 : Nettoyage de la colonne "manufacturing_places".
    df_app.dropna(subset=["manufacturing_places"], inplace=True)
    df_app.loc[
        df_app["manufacturing_places"].str.contains("fr", case=False, na=False),
        ["manufacturing_places"],
    ] = "Made in France"
    df_app.loc[
        ~df_app["manufacturing_places"].str.contains("fr", case=False, na=False),
        ["manufacturing_places"],
    ] = "Not Made in France"
    print("Nettoyage de la colonne `manufacturing_places` OK")

    print("-" * 100)

    # Etape 06 : Imputation des valeurs manquantes sur toutes les colonnes numériques SAUF la colonne "nutri-score"
    print("Etape 06 : Imputation des valeurs manquantes.")

    # Liste des colonnes non concernées par l'imputation
    index_list = [
        "code",
        "url",
        "product_name",
        "pnns_groups_1",
        "pnns_groups_2",
        "countries",
        "manufacturing_places",
        "nutriscore_score",
        "nutriscore_grade",
    ]

    #   Appeler la fonction numerical_Values_Imputation

    df_app_cleaned = numerical_Values_Imputation(df_app, index_list)

    print("-" * 100)

    del df_app

    # Etape 07 : Créer une nouvelle colonne "sodium_100mg", calculer les valeurs de cette colonne à partir de la colonne "salt_10g"
    print("Etape 07 : Création de la colonne sodium_100mg")

    df_app_cleaned["sodium_100mg"] = (df_app_cleaned["salt_100g"].div(2.5)) * 1000
    print("Création de la colonne `sodium_100mg` OK.")

    print("-" * 100)

    # Etape 08 : Calcul des valeurs manquantes sur la colonne "nutri-score" et "nutri-grade"
    print(
        "Etape 08 : Calcul des valeurs manquantes sur les colonnes nutri-score et nutri-grade"
    )

    df_app_cleaned.drop(["nutriscore_score", "nutriscore_grade"], axis=1, inplace=True)

    df_app_cleaned["nutri_score_calculated"] = df_app_cleaned[
        [
            "energy_100g",
            "sugars_100g",
            "saturated-fat_100g",
            "fat_100g",
            "sodium_100mg",
            "proteins_100g",
            "fiber_100g",
            "fruits-vegetables-nuts_100g",
            "pnns_groups_1",
        ]
    ].apply(lambda x: main_Nutri_Score(*x), axis=1)

    print("Imputation des valeurs manquantes sur la colonne nutri-score OK")

    df_app_cleaned["nutri_grade_calculated"] = df_app_cleaned[
        ["pnns_groups_1", "pnns_groups_2", "nutri_score_calculated"]
    ].apply(lambda x: main_Nutri_Grade(*x), axis=1)

    print("Imputation des valeurs manquantes sur la colonne nutri-grade OK")

    # Nous n'avons plus besoin de la colonne countries, car, toutes ces valeurs sont égales à France
    df_app_cleaned = df_app_cleaned.drop(["countries"], axis=1)

    print("-" * 100)

    # Etape 09 : Enregistrer le dataframe nettoyé
    print("Etape 09 : Enregistrement du dataframe nettoyé")

    df_app_cleaned.to_csv(out_file_path, index=False)

    print("Le dataframe nettoyé est nommé :df_cleaned.csv ")
    print("-" * 100)

    return print(
        "Toutes les étapes du nettoyage du dataset ont été réalisé avec succèss "
    )

### <img src='./images/logo_OC.png' width=20px align="left"/> &emsp; Test de la fonction de  nettoyage du dataset.

In [131]:
input_file_path = "data/en.openfoodfacts.org.products.csv"

In [132]:
out_file_path = "data/df_cleaned.csv"

In [133]:
%%time
dataset_auto_clean(input_file_path, out_file_path)

----------------------------------------------------------------------------------------------------
Etape 01 : Charger le dataset.
le dataset a été correctement chargé.
----------------------------------------------------------------------------------------------------
Etape 02 : Définir les bons Dtypes pour les différentes colonnes
Aucune conversion n'est nécessaire des Dtypes.
----------------------------------------------------------------------------------------------------
Etape 03 : Chercher et corriger les Fake NaN
Aucune valeur Fake NaN n'a été détectée
----------------------------------------------------------------------------------------------------
Etape 04 : Suppression des valeurs en double sur la colonne code.
Le nombre de valeurs en double détectées sur le colonne code AVANT correction :  6
Le nombre de valeurs en double détectées sur le colonne code APRES correction :  0
--------------------------------------------------------------------------------------------------

### <img src='./images/logo_OC.png' width=20px align="left"/> &emsp; Exploration du dataset cleané..

In [134]:
df_cleaned = pd.read_csv("data/df_cleaned.csv")

In [135]:
df_cleaned.head()

,code,url,product_name,pnns_groups_1,pnns_groups_2,manufacturing_places,additives_n,energy_100g,fat_100g,saturated-fat_100g,sugars_100g,fiber_100g,proteins_100g,salt_100g,fruits-vegetables-nuts_100g,sodium_100mg,nutri_score_calculated,nutri_grade_calculated
0,0000000005,http://world-en.openfoodfacts.org/product/0000...,Nectar d'abricot,Beverages,Sweetened beverages,Not Made in France,0.0,1598.0,45.50,27.5,2.80,16.445,16.93,1.3,75.0,520.0,18,e
1,0000000020114,http://world-en.openfoodfacts.org/product/0000...,Naturablue original,unknown,unknown,Made in France,1.0,0.0,0.00,0.0,0.00,0.000,0.00,0.0,0.0,0.0,0,b
2,0000000274722,http://world-en.openfoodfacts.org/product/0000...,Blanquette de Volaille et son Riz,Composite foods,One-dish meals,Made in France,2.0,450.0,2.20,0.9,0.50,0.500,6.80,0.7,57.0,280.0,-1,a
3,0000007730009,http://world-en.openfoodfacts.org/product/0000...,Biscuits sablés fourrage au cacao,Sugary snacks,Biscuits and cakes,Made in France,3.0,1866.0,20.45,10.5,36.85,2.350,6.75,0.7,35.0,280.0,24,e
4,0000009336247,http://world-en.openfoodfacts.org/product/0000...,Bonbons acidulés Raisin Fraise,Sugary snacks,Sweets,Not Made in France,7.0,1674.0,0.00,0.0,93.30,0.000,0.00,0.0,32.0,0.0,14,d


In [136]:
df_cleaned.dtypes.sort_values()

nutri_score_calculated           int64
fat_100g                       float64
sodium_100mg                   float64
fruits-vegetables-nuts_100g    float64
salt_100g                      float64
proteins_100g                  float64
fiber_100g                     float64
additives_n                    float64
energy_100g                    float64
saturated-fat_100g             float64
sugars_100g                    float64
code                            object
pnns_groups_2                   object
pnns_groups_1                   object
product_name                    object
url                             object
manufacturing_places            object
nutri_grade_calculated          object
dtype: object

In [137]:
df_cleaned.shape

(59237, 18)

In [138]:
df_cleaned.isna().sum().sort_values(ascending=False)

product_name                   313
pnns_groups_1                    1
code                             0
sugars_100g                      0
nutri_score_calculated           0
sodium_100mg                     0
fruits-vegetables-nuts_100g      0
salt_100g                        0
proteins_100g                    0
fiber_100g                       0
saturated-fat_100g               0
url                              0
fat_100g                         0
energy_100g                      0
additives_n                      0
manufacturing_places             0
pnns_groups_2                    0
nutri_grade_calculated           0
dtype: int64